In [1]:
import ibis

In [2]:
import ibis
from ibis import _
conn = ibis.duckdb.connect("tmp2", extensions=["spatial"])

tbl = (
    conn.read_parquet("https://data.source.coop/cboettig/ca30x30/ca_areas.parquet")
    .cast({"SHAPE": "geometry"})
    .rename(geom = "SHAPE", gid = "OBJECTID")
   # .filter(_.UNIT_NAME == "Angeles National Forest")
    .filter(_.reGAP < 3) 
)
lvl = "Federal"
conn.create_table("t1", tbl.filter(_.Release_Year == 2024, _.MNG_AG_LEV != lvl), overwrite = True)
conn.create_table("t2", tbl.filter(_.Release_Year == 2023, _.MNG_AG_LEV != lvl), overwrite = True)

conn.disconnect()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
%%time
import duckdb
db = duckdb.connect("tmp2")
db.install_extension("spatial")
db.load_extension("spatial")

db.sql('''
CREATE OR REPLACE TABLE diff AS (
with temp as 
(
  select   b.gid, st_union_agg(a.geom) as geom
  from     t1 b join t2 a on st_intersects(a.geom, b.geom)
  group by b.gid
) 
select st_difference(b.geom,coalesce(t.geom, 'GEOMETRYCOLLECTION EMPTY'::geometry)) as geom
from t1 b left join temp t on b.gid = t.gid
)
''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 1min 28s, sys: 6.91 s, total: 1min 35s
Wall time: 1min 28s


In [ ]:
#db.table("diff").to_parquet("diff.parquet")

In [4]:
conn = ibis.duckdb.connect("tmp2", extensions=["spatial"])
gdf = conn.table("diff").mutate(geom = _.geom.convert("epsg:3310","epsg:4326")).execute()

In [5]:
import leafmap.maplibregl as leafmap
m = leafmap.Map()
m.add_gdf(gdf)
m


Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': 'https://basemaps.carto…

In [ ]:
"TO 'new2024.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')"